# Using SQL Render

Examples followed and combined: https://github.com/OHDSI/SqlRender/ and https://ohdsi.github.io/TheBookOfOhdsi/SqlAndR.html

by Anja van Gestel

In [ ]:
## in R:
# remotes::install_github("OHDSI/SqlRender")

%load_ext autoreload
%autoreload 2

import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.3.1" # change as needed
from pathlib import Path

from ohdsi import sqlrender
from ohdsi import database_connector

# Introduction
Sqlrender is a package for rendering parameterized SQL, and translating it to different SQL dialects. 

# Features
* Supports a simple markup syntax for making SQL parameterized, and renders parameterized SQL (containing the markup syntax) to executable SQL
* The syntax supports defining default parameter values
* The syntax supports if-then-else structures
* Has functions for translating SQL from one dialect (Microsoft SQL Server) to other dialects (Oracle, PostgreSQL, Amazon RedShift, Impala, IBM Netezza, Google BigQuery, Microsoft PDW, Snowflake, Azure Synapse, Apache Spark and SQLite)
* Can be used as R package, Java library, or as stand-alone executable through a command-line interface

# List supported dialects

In [ ]:
sqlrender.list_supported_dialects()

# SQL Parameterization

One of the functions of the package is to support parameterization of SQL. Often, small variations of SQL need to be generated based on some parameters. SqlRender offers a simple markup syntax inside the SQL code to allow parameterization. Rendering the SQL based on parameter values is done using the <code>render()</code> function.

## Substituting Parameter Values

The <code>@</code> character can be used to indicate parameter names that need to be exchanged for actual parameter values when rendering. In the following example, a variable called <code>a</code> is mentioned in the SQL. In the call to the render function the value of this parameter is defined:

In [ ]:
sql = "SELECT * FROM concept WHERE concept_id = @a;"
print(sqlrender.render(sql, a = 123))

Note that, unlike the parameterization offered by most database management systems, it is just as easy to parameterize table or field names as values:

In [ ]:
sql = "SELECT * FROM @x WHERE person_id = @a;"
print(sqlrender.render(sql, x = "observation", a = 123))

The parameter values can be numbers, strings, booleans, as well as vectors, which are converted to comma-delimited lists:

In [ ]:
sql = "SELECT * FROM concept WHERE concept_id IN (@a);"
print(sqlrender.render(sql, a = [123, 234, 345]))

# If-Then-Else

Sometimes blocks of codes need to be turned on or off based on the values of one or more parameters. This is done using the <code>{Condition} ? {if true} : {if false}</code> syntax. If the <i>condition</i> evaluates to <i>true</i> or 1, the <i>if true</i> block is used, else the <i>if false</i> block is shown (if present).

In [ ]:
sql = "SELECT * FROM cohort {@x} ? {WHERE subject_id = 1}"
print(sqlrender.render(sql, x = False))

In [ ]:
print(sqlrender.render(sql, x = True))

Simple comparisons are also supported:

In [ ]:
sql = "SELECT * FROM cohort {@x == 1} ? {WHERE subject_id = 1};"
print(sqlrender.render(sql, x = 1))

In [ ]:
print(sqlrender.render(sql, x = 2))

As well as the <code>IN</code> operator:

In [ ]:
sql = "SELECT * FROM cohort {@x IN (1,2,3)} ? {WHERE subject_id = 1};"
print(sqlrender.render(sql, x = 2))

In [ ]:
sql = "SELECT * FROM cohort {@x IN (1,2,3)} ? {WHERE subject_id = 1};"
print(sqlrender.render(sql, x = 5))

# Translation to Other SQL Dialects

Another function of the SqlRender package is to translate from OHDSI SQL to other SQL dialects. For example:

In [ ]:
sql = "SELECT TOP 10 * FROM person;"
sqlrender.translate(sql, target_dialect = "postgresql")

In [ ]:
sqlrender.translate(sql, target_dialect = "oracle")

<b>Note</b><br>
There are limits to what SQL functions and constructs can be translated properly, both because only a limited set of translation rules have been implemented in the package, but also some SQL features do not have an equivalent in all dialects. This is the primary reason why OHDSI SQL was developed as its own, new SQL dialect. However, whenever possible we have kept to the SQL Server syntax to avoid reinventing the wheel.

# SQL from file

Reading SQL from file is of course essential as well, so let's take a look.

In [ ]:
sql_file = r'./input/cohorts_of_interest.sql'
sql = sqlrender.read_sql(sql_file)
print(sql)

# SQL &rarr; file

Writing SQL to a file is possible as well.

In [ ]:
sql_file = r'./input/cohorts_of_interest.sql'
sql = sqlrender.read_sql(sql_file)

filename = 'write_sql_to_file.sql'
sqlrender.write_sql(sql, filename)
print(f'New file created: {filename}')

# SQL file &rarr; rendered SQL file

It is also possible to create a new file directly from a SQL file, rendered with all parameters. Just feed the function <code>render_sql_file()</code> with the source file, target file, and all the parameters and their corresponding target values. In this example the SQL file contains text like "@resultsDatabaseSchema" and "@cdmDatabaseSchema". You will have to tell <code>render_sql_file()</code> what values you want to be rendered instead of these parameters.

In [ ]:
sql_file = r'./input/cohorts_of_interest.sql'
sql_file2 = 'rendered_cohorts_of_interest.sql'
sqlrender.render_sql_file(sql_file, sql_file2, 
                          resultsDatabaseSchema = "my_results_schema", 
                          cdmDatabaseSchema = "my_cdm_schema")
print(f'New file created: {sql_file2}')

# SQL file &rarr; translated SQL file

Whenever you want to translate SQL from a file to SQL of a different dialect in a file, <code>translate_sql_file()</code> is your function!

In [ ]:
sql_file = r'./input/cohorts_of_interest.sql'
sql_file2 = 'translated_cohorts_of_interest.sql'
sqlrender.translate_sql_file(sql_file, sql_file2, 'oracle')
print(f'New file created: {sql_file2}')

# Read, Render, Translate, Execute

The Sqlrender package together with the SQL functions from DatabaseConnector make that you can render, translate, and execute SQL scripts on your own database.

### Your own parameters

For these tutorials I am using a PostgreSql database, which I created using Synthea (https://github.com/synthetichealth/synthea). My database is called 'synthea10', it has a schema 'cdm_synthea10' which contains the OMOP CDM tables, and another schema 'results' which I use for results/cohorts/etc.

For the code in this tutorial to work you should of course call your own database / schemas / user / password.

In [ ]:
cdm_database = 'synthea10'
cdm_schema = 'cdm_synthea10'
results_schema = 'results'
user = 'postgres'
password = 'password'

### Connection to the database

In [ ]:
connection_details = database_connector.create_connection_details(
    dbms = "postgresql",
    server = f"localhost/{cdm_database}",
    user = f"{user}",
    password = f"{password}",
    port = 5432)
conn = database_connector.connect(connection_details)

### SQL transformations

Reading, rendering, translating the SQL from a file.

In [ ]:
sql_file = r'./input/cohorts_of_interest.sql'
sql = sqlrender.read_sql(sql_file)
sql = sqlrender.render(sql, cdmDatabaseSchema = cdm_schema, resultsDatabaseSchema = results_schema)
sql = sqlrender.translate(sql, 'postgresql')

### Execute

Executing this SQL creates a 'cohorts_of_interest' table in our results schema in the database!

In [ ]:
database_connector.execute_sql(conn, sql)

### Disconnect

In [ ]:
database_connector.disconnect(conn)

print('Done')

# Epilogue

In this tutorial we handled the most commonly used functions of the Sqlrender, if you are looking for more check out the docs and feel free to play around with <code>translate_single_statement()</code>, <code>get_temp_table_prefix()</code>, <code>split_sql()</code>, and <code>load_render_translate_sql()</code> as well.